Work in progress:
- add tensorboard support (see https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab)
- model save / restore
- retrain / fine-tune network based on pretrained weights

![Toronto AI](https://i.imgur.com/diILtDP.png)

# Neural Networks

A trained neural net can be thought of as a learned mapping.

Here are some examples of mappings that a neural net could learn:

*   Mapping English to French
*   Mapping pictures to text descriptions
*   Mapping live sensor data from a reusable rocket to control commands that land it
*   Mapping random vectors into images of flowers

In essence, we use neural nets to map one distribution of data onto another.

Here's an an example where I trained a neural net to map random vectors onto the space of flower photos, using a Generative Adversarial Network:

![](https://i.imgur.com/SaT9OEM.png)

# Tensors

* Tensors are multidimensional arrays.
* They are like boxes of data, that we use to contain our data, or the weights of our model.
* Tensors are used extensively in TensorFlow to represent:
  * 0-D - scalars
  * 1-D - vectors, text
  * 2-D - matrices, tables of data
  * 3-D - batches of matrices, a cube of data, e.g. an image, a monochrome video
  * 4-D - convolution kernels, a colour video
  * 5-D - batches of colour video
  * 6-D - 3D vector fields
  * 7-D - layered 3D vector fields (e.g. gravity and electromagnetism layers)
  * 8-D - batches of layered 3D vector fields 
  * 9-D - batches of layered 3D vector fields evolving through time
  * keep going...

* GPU memory is expensive, so Tensors are most commonly 4-D or less.

* It helps to visualize a Tensor as a Rubiks Cube - each cell holds a piece of scalar data (like a weight, a piece of input data, or a label).  For higher dimensional Tensors, think of each cell as holding a Tensor instead of a scalar.
![](https://i.imgur.com/KyOQVX9.png)

This notebook is based on Toronto AI - Noise-Circle Synthetic Dataset https://github.com/toronto-ai/noise-circle-dataset, modified for Google Colab environment using Keras.

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2330196015497461093, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11297803469
 locality {
   bus_id: 1
 }
 incarnation: 5422562790259915404
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

Mapping your Google Drive space to Colab


In [0]:
# todo

# use mapped google drive url instead
#import os
#DIRECTORY = os.path.join("~", "ai")

In [0]:
# https://keras.io/
!pip install -q keras

from __future__ import print_function
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.losses import mean_squared_error
from keras.optimizers import  Adam
from keras.callbacks import Callback
from keras import backend as K

In [0]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random, math
from matplotlib import animation, rc
from IPython.display import HTML

In [0]:
plt.rcParams['image.cmap'] = 'viridis'

np.random.seed(20180118)

BATCH_SIZE = 16
DIM = 64
TWO_PI = 2.*math.pi
MIN_CIRCLE_WIDTH = int(DIM/10)
NUM_CIRCLE_DOTS = 500
CIRCLE_WIDTH = 3
DARKNESS = .5

In [0]:
# We'll use this to create our data set.
def add_circles(data):
    radius = int((random.random() * (DIM/2 - MIN_CIRCLE_WIDTH)) + MIN_CIRCLE_WIDTH)
    xpos = random.random()*DIM
    ypos = random.random()*DIM
    
    draw_circle(data, xpos, ypos, radius, DARKNESS, CIRCLE_WIDTH)

    return [xpos, ypos, radius+CIRCLE_WIDTH/2]


def draw_circle(data, xpos, ypos, radius, darkness, width):
    for i in range(NUM_CIRCLE_DOTS):
        for r in range(radius, radius+width):
            rad = TWO_PI * i/NUM_CIRCLE_DOTS
            x = int(round(r*math.cos(rad)+xpos))
            y = int(round(-r*math.sin(rad)+ypos))
            if x >= 0 and x < DIM and y >= 0 and y < DIM:
                data[x,y] = data[x,y] - darkness

# Create random noise and draw circles in it
def create_dataset_row():
    data = np.random.random((DIM, DIM)).astype(np.float32)
    label = add_circles(data)
    label = np.array(label).astype(np.float32)
    
    return (data, label)
    

def create_dataset(rows):
    
    labels  = []
    samples = []
    for i in range(rows):
        data, label = create_dataset_row()
        labels.append(label)
        samples.append(data)
    return (np.array(samples).astype(np.float32), np.array(labels).astype(np.float32))

def plot_dataset(data):
    fig, ax = plt.subplots()

    def init():
        ax.cla()
        return ()

    def animate(i):
        ax.imshow(data[i%len(data)])
        return ()

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=BATCH_SIZE, interval=700, blit=True)
    plt.close(fig)
    fig.set_size_inches(10, 10, True)
    return HTML(anim.to_jshtml())


In [12]:
# Create an animation so we can see our data set
dataset = create_dataset(BATCH_SIZE)
data   = dataset[0]
labels = dataset[1]

plot_dataset(data)

Build the Keras Model here

input: BATCH_SIZE x 64 x 64 x 1

TRUNK:
C1 - CONV: 32 filters, 7x7 kernal, relu
C2 - CONV: 32 filters, 7x7 kernal, relu
C3 - CONV: 32 filters, 7x7 kernal, relu

C4 - CONV: 32 filters, 7x7 kernal, relu
A4 - AVG POOLING: 2x2, strides 2

C5 - CONV: 32 filters, 7x7 kernal, relu 
A5 - AVG POOLING: 2x2, strides 2

C6 - CONV: 32 filters, 7x7 kernal, relu 
A6 - AVG POOLING: 2x2, strides 2

output': BATCH_SIZE x 8 x 8 x 32

HEAD (3x, one for xpos, one for ypos and one for radius)

D7x - DENSE: 32, elu
D8x - DENSE: 32, elu
D9x - DENSE: 32, elu

output: sum(D7x,D8x,D9x)

In [0]:
# Setup the input to the network

# check the CPU/GPU image convention
# based on keras mnist example
if K.image_data_format() == 'channels_first':
    input_shape = (1, DIM, DIM)
else:
    input_shape = (DIM, DIM, 1)
    
#main_input = Input(shape=input_shape, dtype='float32', name='main_input')

# Batch version
main_input = Input(batch_shape=(BATCH_SIZE,)+input_shape, dtype='float32', name='main_input')


In [0]:
# number of filters per conv2d layers
FS = 32

# conv2d kernel window size
KerSz  = 7

#Ref:
# keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', 
#   data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, 
#   kernel_initializer='glorot_uniform', bias_initializer='zeros', 
#   kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
#   kernel_constraint=None, bias_constraint=None)
conv = Conv2D(FS, (KerSz, KerSz), padding='same',activation='relu',kernel_initializer='glorot_normal')

# keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)
avgp = AveragePooling2D(pool_size=(2, 2), strides=(2,2))
maxp = MaxPooling2D(pool_size=(2, 2), strides=(2,2))


In [0]:
# C1
# without the input_shape at first conv2d it won't work!
x = Conv2D(FS, (KerSz, KerSz), padding='same',activation='relu', 
           input_shape=input_shape)(main_input)

x = conv(x) # C2
x = conv(x) # C3

x = conv(x) # C4
x = avgp(x) # A4 (swap out with maxp if you want)

x = conv(x) # C5
x = avgp(x) # A5 (swap out with maxp if you want)

x = conv(x) # C6
x = avgp(x) # A6 (swap out with maxp if you want)

# flattern the conv2d layer output to make it a table, one row for each example 
# in the batch.
xf = Flatten()(x)

In [0]:
# make lambda layers between the trunk and head
# https://keras.io/layers/core/
# stackoverflow: https://github.com/keras-team/keras/issues/890
# keras.layers.Lambda(function, output_shape=None, mask=None, arguments=None)
#
# split the conv2d layer output for x,y,r estimation branches

#a_third  = int(x.shape[1].value / 3)
a_third = 682 # hard coded (TODO: fix this!!!)

xx = Lambda(lambda x: x[:,0*a_third:(0+1)*a_third])(xf)
yy = Lambda(lambda x: x[:,1*a_third:(1+1)*a_third])(xf)
rr = Lambda(lambda x: x[:,2*a_third:(2+1)*a_third])(xf)

In [0]:
# feed the input to the sub networks for regression (e.g.x_guess, y_guess etc.)

# number of neurons in the hidden layers
DS = 32 
af = 'elu'

# Reference:
# keras.layers.Dense(units, activation=None, use_bias=True, 
#   kernel_initializer='glorot_uniform', bias_initializer='zeros', 
#   kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
#   kernel_constraint=None, bias_constraint=None)

def denseSubnet(name,input):
  layer = Dense(DS,activation=af,kernel_initializer='glorot_normal')(input)
  layer = Dense(DS,activation=af,kernel_initializer='glorot_normal')(layer)
  layer = Dense(DS,activation=af,kernel_initializer='glorot_normal')(layer)
  # instead of using sum (which doesn't work here)
  #   return Lambda(lambda x: K.sum(x, axis=1))(layer)
  layer = Dense(1,activation='linear',name=name)(layer) 
  return layer

x_guess = denseSubnet('x_guess',xx)
y_guess = denseSubnet('y_guess',yy)
r_guess = denseSubnet('r_guess',rr)

# try without spliting up the conv2d output later
#x_guess = denseSubnet('x_guess',xf)
#y_guess = denseSubnet('y_guess',xf)
#r_guess = denseSubnet('r_guess',xf)


In [18]:
model = Model(inputs=[main_input], outputs=[x_guess,y_guess,r_guess])

# display the model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (16, 64, 64, 1)      0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (16, 64, 64, 32)     1600        main_input[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               multiple             50208       conv2d_2[0][0]                   
                                                                 conv2d_1[0][0]                   
                                                                 conv2d_1[1][0]                   
                                                                 average_pooling2d_1[0][0]        
          

Define the custom loss and metric for the network, required for the model.fit in Keras

TODO: The metric and loss output doesn't make sense

In [0]:
# Ref:
# https://github.com/keras-team/keras/blob/master/keras/losses.py
def loss_metric_helper(y_true, y_pred):
  actual_x, actual_y, actual_r = y_true[0], y_true[1], y_true[2]
  x,y,r = y_pred[0], y_pred[1], y_pred[2]
  
  # Define an error between the predicted x, y and radius, and their actual values
  # We will later ask the network to minimize this error through gradient descent
  err = mean_squared_error(y_true, y_pred)
  x_error, y_error, r_error = err[0],err[1],err[2]
  
  return x_error, y_error, r_error

def loss_function(y_true, y_pred):
  x_error, y_error, r_error = loss_metric_helper(y_true, y_pred)
  
  # Let's combine these individual loss metrics into a combined loss function
  # Note: these are vectors - with separate losses defined for each element in the batch 
  return x_error + y_error + r_error

def avg_dist_metric(y_true, y_pred):
  x_error, y_error, _ = loss_metric_helper(y_true, y_pred)
    
  # Now lets create some metrics that we can use to evaluate our progress
  avg_distance_from_actual_center = x_error + y_error
  return avg_distance_from_actual_center

def avg_radius_error_metric(y_true, y_pred):
  _, _, r_error = loss_metric_helper(y_true, y_pred)
  avg_radius_error = r_error
  return avg_radius_error

In [0]:
# setup the optimizer
# ref: https://keras.io/optimizers/
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False

ALPHA = 0.001
BETA1 = 0.9
BETA2 = 0.999
EPSILON=None #1e-9

opt = Adam(lr=ALPHA,beta_1=BETA1,beta_2=BETA2,epsilon=EPSILON)


In [0]:
# try to compile the model
model.compile(loss=loss_function, 
              optimizer=opt,
              metrics=[avg_dist_metric, avg_radius_error_metric])

Define the batch sample generator to generate an infinite dataset in Keras convention

Ref:
https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly.html


In [0]:
class DataGenerator(object):
  def __init__(self, dim_x = 32, dim_y = 32, dim_z = 32, batch_size = 32, shuffle = True):
    'Initialization'
    self.dim_x = dim_x
    self.dim_y = dim_y
    self.batch_size = batch_size
    self.shuffle = shuffle
    
  def generate(self):
    'Generates batches of samples'
    # Infinite loop
    while 1:
      # Generate order of exploration of dataset
      samples = np.zeros((BATCH_SIZE,DIM,DIM,1))
      ds = create_dataset(BATCH_SIZE)
      samples[:,:,:,0] = ds[0]
      labels = ds[1]
      yield (samples,[labels[:,0],labels[:,1],labels[:,2]])

  def generateOne(self):
    # one sample batch output
    samples = np.zeros((BATCH_SIZE,DIM,DIM,1))
    ds = create_dataset(BATCH_SIZE)
    samples[:,:,:,0] = ds[0]
    labels = ds[1]
    return (samples,[labels[:,0],labels[:,1],labels[:,2]])
      
# Parameters
params = {'dim_x': DIM,
          'dim_y': DIM,
          'batch_size': BATCH_SIZE,
          'shuffle': False}

generator = DataGenerator(**params).generate()      

Define callbacks for misc. purpose


In [0]:
# Ref:
# https://keras.io/callbacks/#example-recording-loss-history
class LossHistory(keras.callbacks.Callback):
  def __init__(self):
    self.idx = 0
    
  def on_train_begin(self, logs={}):
    self.losses = []

  def on_batch_end(self, batch, logs={}):
    #self.losses.append(logs.get('metric'))  
    xErr = logs.get('x_guess_avg_dist_metric')
    yErr = logs.get('y_guess_avg_dist_metric')
    rErr = logs.get('r_guess_avg_radius_error_metric')
    
    ema_radius = rErr
    ema_center = (xErr + yErr)/2

    print("step: {step}, \
      EMA radius error: {ema_radius:.0f}px. \
      EMA center error: {ema_center:.0f}px".format(step=self.idx, 
                                                   ema_radius=ema_radius, 
                                                   ema_center=ema_center))
    self.idx += 1
  
history = LossHistory()


train the model with generator

In [64]:
np.random.seed(20180118)
model.fit_generator(generator = generator,
                    steps_per_epoch=1, epochs=1000,
                    workers = 0 , # This is important
                    verbose = 0,
                    callbacks=[history])

step: 0,       EMA radius error: 0px.       EMA center error: 36px
step: 1,       EMA radius error: 0px.       EMA center error: 36px
step: 2,       EMA radius error: 5px.       EMA center error: 34px
step: 3,       EMA radius error: 3px.       EMA center error: 100px
step: 4,       EMA radius error: 0px.       EMA center error: 33px
step: 5,       EMA radius error: 5px.       EMA center error: 14px
step: 6,       EMA radius error: 9px.       EMA center error: 9px
step: 7,       EMA radius error: 28px.       EMA center error: 59px
step: 8,       EMA radius error: 10px.       EMA center error: 24px
step: 9,       EMA radius error: 0px.       EMA center error: 38px
step: 10,       EMA radius error: 4px.       EMA center error: 26px
step: 11,       EMA radius error: 0px.       EMA center error: 13px
step: 12,       EMA radius error: 0px.       EMA center error: 4px
step: 13,       EMA radius error: 11px.       EMA center error: 13px
step: 14,       EMA radius error: 1px.       EMA center 

step: 74,       EMA radius error: 1px.       EMA center error: 9px
step: 75,       EMA radius error: 0px.       EMA center error: 3px
step: 76,       EMA radius error: 1px.       EMA center error: 2px
step: 77,       EMA radius error: 0px.       EMA center error: 6px
step: 78,       EMA radius error: 1px.       EMA center error: 1px
step: 79,       EMA radius error: 2px.       EMA center error: 7px
step: 80,       EMA radius error: 2px.       EMA center error: 5px
step: 81,       EMA radius error: 0px.       EMA center error: 6px
step: 82,       EMA radius error: 0px.       EMA center error: 4px
step: 83,       EMA radius error: 1px.       EMA center error: 19px
step: 84,       EMA radius error: 7px.       EMA center error: 2px
step: 85,       EMA radius error: 1px.       EMA center error: 0px
step: 86,       EMA radius error: 0px.       EMA center error: 6px
step: 87,       EMA radius error: 1px.       EMA center error: 1px
step: 88,       EMA radius error: 2px.       EMA center error

step: 148,       EMA radius error: 0px.       EMA center error: 1px
step: 149,       EMA radius error: 1px.       EMA center error: 5px
step: 150,       EMA radius error: 1px.       EMA center error: 13px
step: 151,       EMA radius error: 1px.       EMA center error: 9px
step: 152,       EMA radius error: 1px.       EMA center error: 5px
step: 153,       EMA radius error: 0px.       EMA center error: 4px
step: 154,       EMA radius error: 0px.       EMA center error: 4px
step: 155,       EMA radius error: 0px.       EMA center error: 3px
step: 156,       EMA radius error: 0px.       EMA center error: 0px
step: 157,       EMA radius error: 0px.       EMA center error: 2px
step: 158,       EMA radius error: 0px.       EMA center error: 5px
step: 159,       EMA radius error: 1px.       EMA center error: 2px
step: 160,       EMA radius error: 3px.       EMA center error: 1px
step: 161,       EMA radius error: 1px.       EMA center error: 3px
step: 162,       EMA radius error: 5px.       E

step: 222,       EMA radius error: 2px.       EMA center error: 3px
step: 223,       EMA radius error: 1px.       EMA center error: 9px
step: 224,       EMA radius error: 0px.       EMA center error: 2px
step: 225,       EMA radius error: 0px.       EMA center error: 3px
step: 226,       EMA radius error: 4px.       EMA center error: 3px
step: 227,       EMA radius error: 3px.       EMA center error: 4px
step: 228,       EMA radius error: 0px.       EMA center error: 13px
step: 229,       EMA radius error: 1px.       EMA center error: 2px
step: 230,       EMA radius error: 1px.       EMA center error: 0px
step: 231,       EMA radius error: 1px.       EMA center error: 15px
step: 232,       EMA radius error: 0px.       EMA center error: 4px
step: 233,       EMA radius error: 0px.       EMA center error: 2px
step: 234,       EMA radius error: 1px.       EMA center error: 1px
step: 235,       EMA radius error: 13px.       EMA center error: 9px
step: 236,       EMA radius error: 2px.      

step: 296,       EMA radius error: 0px.       EMA center error: 2px
step: 297,       EMA radius error: 0px.       EMA center error: 4px
step: 298,       EMA radius error: 0px.       EMA center error: 12px
step: 299,       EMA radius error: 7px.       EMA center error: 3px
step: 300,       EMA radius error: 3px.       EMA center error: 2px
step: 301,       EMA radius error: 0px.       EMA center error: 2px
step: 302,       EMA radius error: 0px.       EMA center error: 6px
step: 303,       EMA radius error: 2px.       EMA center error: 2px
step: 304,       EMA radius error: 1px.       EMA center error: 7px
step: 305,       EMA radius error: 0px.       EMA center error: 8px
step: 306,       EMA radius error: 3px.       EMA center error: 2px
step: 307,       EMA radius error: 0px.       EMA center error: 8px
step: 308,       EMA radius error: 0px.       EMA center error: 15px
step: 309,       EMA radius error: 7px.       EMA center error: 5px
step: 310,       EMA radius error: 8px.       

...

step: 666,       EMA radius error: 1px.       EMA center error: 3px
step: 667,       EMA radius error: 2px.       EMA center error: 3px
step: 668,       EMA radius error: 1px.       EMA center error: 1px
step: 669,       EMA radius error: 0px.       EMA center error: 4px
step: 670,       EMA radius error: 3px.       EMA center error: 1px
step: 671,       EMA radius error: 3px.       EMA center error: 0px
step: 672,       EMA radius error: 1px.       EMA center error: 10px
step: 673,       EMA radius error: 0px.       EMA center error: 4px
step: 674,       EMA radius error: 0px.       EMA center error: 2px
step: 675,       EMA radius error: 1px.       EMA center error: 0px
step: 676,       EMA radius error: 2px.       EMA center error: 1px
step: 677,       EMA radius error: 0px.       EMA center error: 2px
step: 678,       EMA radius error: 0px.       EMA center error: 2px
step: 679,       EMA radius error: 1px.       EMA center error: 1px
step: 680,       EMA radius error: 3px.       E

step: 741,       EMA radius error: 0px.       EMA center error: 3px
step: 742,       EMA radius error: 1px.       EMA center error: 1px
step: 743,       EMA radius error: 0px.       EMA center error: 1px
step: 744,       EMA radius error: 2px.       EMA center error: 1px
step: 745,       EMA radius error: 1px.       EMA center error: 1px
step: 746,       EMA radius error: 0px.       EMA center error: 5px
step: 747,       EMA radius error: 0px.       EMA center error: 0px
step: 748,       EMA radius error: 0px.       EMA center error: 4px
step: 749,       EMA radius error: 0px.       EMA center error: 5px
step: 750,       EMA radius error: 2px.       EMA center error: 1px
step: 751,       EMA radius error: 1px.       EMA center error: 1px
step: 752,       EMA radius error: 0px.       EMA center error: 1px
step: 753,       EMA radius error: 0px.       EMA center error: 3px
step: 754,       EMA radius error: 0px.       EMA center error: 1px
step: 755,       EMA radius error: 0px.       EM

step: 816,       EMA radius error: 1px.       EMA center error: 3px
step: 817,       EMA radius error: 2px.       EMA center error: 2px
step: 818,       EMA radius error: 48px.       EMA center error: 1px
step: 819,       EMA radius error: 0px.       EMA center error: 1px
step: 820,       EMA radius error: 0px.       EMA center error: 2px
step: 821,       EMA radius error: 0px.       EMA center error: 0px
step: 822,       EMA radius error: 1px.       EMA center error: 1px
step: 823,       EMA radius error: 0px.       EMA center error: 8px
step: 824,       EMA radius error: 1px.       EMA center error: 0px
step: 825,       EMA radius error: 1px.       EMA center error: 0px
step: 826,       EMA radius error: 7px.       EMA center error: 1px
step: 827,       EMA radius error: 1px.       EMA center error: 2px
step: 828,       EMA radius error: 1px.       EMA center error: 11px
step: 829,       EMA radius error: 0px.       EMA center error: 8px
step: 830,       EMA radius error: 0px.       

step: 890,       EMA radius error: 0px.       EMA center error: 1px
step: 891,       EMA radius error: 0px.       EMA center error: 5px
step: 892,       EMA radius error: 0px.       EMA center error: 1px
step: 893,       EMA radius error: 1px.       EMA center error: 1px
step: 894,       EMA radius error: 0px.       EMA center error: 1px
step: 895,       EMA radius error: 0px.       EMA center error: 1px
step: 896,       EMA radius error: 0px.       EMA center error: 2px
step: 897,       EMA radius error: 0px.       EMA center error: 3px
step: 898,       EMA radius error: 1px.       EMA center error: 2px
step: 899,       EMA radius error: 1px.       EMA center error: 2px
step: 900,       EMA radius error: 0px.       EMA center error: 1px
step: 901,       EMA radius error: 1px.       EMA center error: 1px
step: 902,       EMA radius error: 1px.       EMA center error: 7px
step: 903,       EMA radius error: 0px.       EMA center error: 1px
step: 904,       EMA radius error: 0px.       EM

step: 964,       EMA radius error: 1px.       EMA center error: 1px
step: 965,       EMA radius error: 0px.       EMA center error: 2px
step: 966,       EMA radius error: 0px.       EMA center error: 0px
step: 967,       EMA radius error: 2px.       EMA center error: 2px
step: 968,       EMA radius error: 1px.       EMA center error: 0px
step: 969,       EMA radius error: 1px.       EMA center error: 0px
step: 970,       EMA radius error: 1px.       EMA center error: 6px
step: 971,       EMA radius error: 1px.       EMA center error: 2px
step: 972,       EMA radius error: 2px.       EMA center error: 1px
step: 973,       EMA radius error: 0px.       EMA center error: 2px
step: 974,       EMA radius error: 0px.       EMA center error: 2px
step: 975,       EMA radius error: 0px.       EMA center error: 2px
step: 976,       EMA radius error: 0px.       EMA center error: 4px
step: 977,       EMA radius error: 0px.       EMA center error: 2px
step: 978,       EMA radius error: 0px.       EM

Evaluate the model and perform inferrence

In [65]:
sample_batch, actual = DataGenerator(**params).generateOne()
prediction = model.predict(sample_batch,batch_size=BATCH_SIZE) # this seems to work ...

xp = prediction[0].reshape(-1,).astype(np.int32)
yp = prediction[1].reshape(-1,).astype(np.int32)
rp = prediction[2].reshape(-1,).astype(np.int32)
xa, ya, ra      = [np.rint(a).astype(np.int32) for a in actual]

result_display = []

# Add the prediction and the actual to the dataset
for i, sample in enumerate(sample_batch):
    draw_circle(sample, xa[i], ya[i], ra[i],  DARKNESS, CIRCLE_WIDTH)
    draw_circle(sample, xp[i], yp[i], rp[i], -DARKNESS, CIRCLE_WIDTH)  # The prediction will appear bright
    result_display.append(sample)

plot_dataset(np.array(result_display)[:,:,:,0])